# 원핫인코딩 (One-hot Encoding)

- 의미: 단어(또는 토큰)를 어휘 크기(vocab size)만큼의 길이를 가진 벡터로 바꾸는 방법
- 표현 방식: 해당 단어의 인덱스 위치만 1, 나머지는 전부 0 (즉, “이 단어가 맞다/아니다”만 표시)
- 예시: vocab size가 5이고 단어 인덱스가 3이면 → [0, 0, 1, 0, 0]
- 특징/주의: 단어 간 의미/유사도 정보는 없고, vocab이 커질수록 벡터가 매우 커져 메모리/연산 비용이 증가함
	- 그래서 실무에선 보통 임베딩(Embedding)으로 저차원 밀집(dense) 벡터로 바꿔 사용한다
- 사용 시기: 단어 수가 아주 작거나(카테고리/키워드 몇십~몇백) 간단한 모델/해석이 중요한 경우엔 원-핫/BoW를 사용할 때도 있다.

- 임베딩
	- 학습 가능한 변환(레이어/행렬): 단어 인덱스 k를 길이 d(예: 64, 128)인 밀집(dense) 벡터로 매핑
	- 예: k=3 → [0.12, -0.03, ...] (d차원)
	- 특징: 저차원, 밀집, 학습을 통해 의미/유사도가 반영될 수 있음

In [1]:
raw_text = """The Little Prince, written by Antoine de Saint-Exupéry, is a poetic tale about a young prince who travels from his home planet to Earth. The story begins with a pilot stranded in the Sahara Desert after his plane crashes. While trying to fix his plane, he meets a mysterious young boy, the Little Prince.

The Little Prince comes from a small asteroid called B-612, where he lives alone with a rose that he loves deeply. He recounts his journey to the pilot, describing his visits to several other planets. Each planet is inhabited by a different character, such as a king, a vain man, a drunkard, a businessman, a geographer, and a fox. Through these encounters, the Prince learns valuable lessons about love, responsibility, and the nature of adult behavior.

On Earth, the Little Prince meets various creatures, including a fox, who teaches him about relationships and the importance of taming, which means building ties with others. The fox's famous line, "You become responsible, forever, for what you have tamed," resonates with the Prince's feelings for his rose.

Ultimately, the Little Prince realizes that the essence of life is often invisible and can only be seen with the heart. After sharing his wisdom with the pilot, he prepares to return to his asteroid and his beloved rose. The story concludes with the pilot reflecting on the lessons learned from the Little Prince and the enduring impact of their friendship.

The narrative is a beautifully simple yet profound exploration of love, loss, and the importance of seeing beyond the surface of things."""

In [2]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

sentences = sent_tokenize(raw_text)    # 원문을 문장 리스트로 분리

en_stopwords = stopwords.words('english')    # 영어 불용어 목록

vocab = {}    # 단어사전 (key=단어, value=빈도)
preprocessed_sentences = []    # 문장별 전처리 토큰 리스트

for sentence in sentences:
    sentence = sentence.lower()                                          # 소문자 변환
    tokens = word_tokenize(sentence)                                     # 문장을 단어(토큰) 리스트로 변환
    tokens = [token for token in tokens if token not in en_stopwords]    # 불용어 토큰 제거
    tokens = [token for token in tokens if len(token) > 2]               # 길이 2 이하 토큰 제거 (노이즈 감소)

    # 전처리된 토큰들로 빈도 집계 
    for token in tokens:
        # token이 vocab에 없으면 빈도 1로 초기화, vocab에 이미 존재하면 +1
        if token not in vocab:
            vocab[token] = 1
        else:
            vocab[token] += 1

    preprocessed_sentences.append(tokens)

print(preprocessed_sentences)
print(vocab)

[['little', 'prince', 'written', 'antoine', 'saint-exupéry', 'poetic', 'tale', 'young', 'prince', 'travels', 'home', 'planet', 'earth'], ['story', 'begins', 'pilot', 'stranded', 'sahara', 'desert', 'plane', 'crashes'], ['trying', 'fix', 'plane', 'meets', 'mysterious', 'young', 'boy', 'little', 'prince'], ['little', 'prince', 'comes', 'small', 'asteroid', 'called', 'b-612', 'lives', 'alone', 'rose', 'loves', 'deeply'], ['recounts', 'journey', 'pilot', 'describing', 'visits', 'several', 'planets'], ['planet', 'inhabited', 'different', 'character', 'king', 'vain', 'man', 'drunkard', 'businessman', 'geographer', 'fox'], ['encounters', 'prince', 'learns', 'valuable', 'lessons', 'love', 'responsibility', 'nature', 'adult', 'behavior'], ['earth', 'little', 'prince', 'meets', 'various', 'creatures', 'including', 'fox', 'teaches', 'relationships', 'importance', 'taming', 'means', 'building', 'ties', 'others'], ['fox', 'famous', 'line', 'become', 'responsible', 'forever', 'tamed', 'resonates', '

In [3]:
# keras 토큰 인덱싱 + 정수 시퀀스 변환 + padding/trucation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=15, oov_token='<OOV>')     # 객체 생성 (상위 15개 단어, <OOV> 토큰 사용)
tokenizer.fit_on_texts(preprocessed_sentences)             # 학습 및 단어사전 생성
sequences = tokenizer.texts_to_sequences(preprocessed_sentences)     # 문장(토큰 리스트) -> 정수 인덱스 시퀀스로 변환

# 길이 10으로 통일. 길면 앞을 자르고, 짧으면 앞을 0으로 채우기
padded_seqs = pad_sequences(sequences, maxlen=10, truncating='pre', padding='pre')
padded_seqs

array([[ 1,  1,  1,  1,  7,  2,  1,  1,  8,  9],
       [ 0,  0, 10,  1,  4,  1,  1,  1, 11,  1],
       [ 0,  1,  1, 11, 12,  1,  7,  1,  3,  2],
       [ 1,  1, 13,  1,  1,  1,  1,  5,  1,  1],
       [ 0,  0,  0,  1,  1,  4,  1,  1,  1,  1],
       [ 1,  1,  1,  1,  1,  1,  1,  1,  1,  6],
       [ 1,  2,  1,  1, 14,  1,  1,  1,  1,  1],
       [ 1,  6,  1,  1,  1,  1,  1,  1,  1,  1],
       [ 1,  1,  1,  1,  1,  1,  1,  2,  1,  5],
       [ 1,  3,  2,  1,  1,  1,  1,  1,  1,  1],
       [ 0,  0,  1,  1,  4,  1,  1, 13,  1,  5],
       [ 1,  4,  1, 14,  1,  3,  2,  1,  1,  1],
       [ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1]], dtype=int32)

In [ ]:
padded_seqs.shape    # (문장 수, 길이)

(13, 10)

In [ ]:
# 정수 시퀀스를 원-핫 벡터로 변환 (One-hot Encoding)
from tensorflow.keras.utils import to_categorical    # 정수 라벨/인덱스를 원-핫 벡터로 바꾸는 함수

one_hot_encodded = to_categorical(padded_seqs)       # (문장, 길이) 정수 인덱스를 (문장, 길이, 클래스 수) 원-핫으로 변환
one_hot_encodded

array([[[0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.]],

       [[1., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]],

       ...,

       [[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [6]:
one_hot_encodded.shape

(13, 10, 15)

shape이 (문장 개수, 토큰길이, vocab_size)  
마지막 차원인 vocab_size는 padded_seqs에 등장한 최대 인덱스 + 1 크기이다. (0=PAD 포함)

### 한국어 전처리

1. 토큰화 (형태소 분석)
2. 시퀀스 처리 Tokenizer
3. 패딩처리 pad_sequences
4. one-hot encoding

In [8]:
texts = [
    "나는 오늘 학원에 간다.",
    "친구들과 맛있는 점심 식사를 해서 기분이 좋다.",
    "오늘은 또 어떤 재미난 수업을 할지 너무 기대된다."
]

In [10]:
# KoNLPy(Okt) 형태소 분석 기반 한국어 전처리 + 불용어/기호 제거
from konlpy.tag import Okt    # KoNLPy의 Okt 형태소 분석기
import re                     # 정규표현식

okt = Okt()    # Okt 형태소 분석기 객체 생성

# 한국어 불용어 리스트
ko_stopwords = ["은", "는", "이", "가", "을", "를", "와", "과", "에", "의", "으로", "나", "내", "우리", "들", "또", "난", ]

preprocessed_texts = []  # 전처리 토큰 결과 저장용

for text in texts:    # 원문 텍스트 리스트를 한 문장씩 ㅅ둔회
    tokens = okt.morphs(text, stem=True)    # 형태소 단위로 분해(어간 추출 적용)
    tokens = [token for token in tokens if token not in ko_stopwords]    # 불용어 제거
    tokens = [token for token in tokens if not re.search(r'[\s.,:;?!]', token)]    # 공백/구두점 기호가 포함된 토큰 제거
    preprocessed_texts.append(tokens)    # 전처리된 결과를 저장

preprocessed_texts

[['오늘', '학원', '간다'],
 ['친구', '맛있다', '점심', '식사', '하다', '기분', '좋다'],
 ['오늘', '어떻다', '재미', '수업', '하다', '너무', '기대', '되다']]

In [12]:
# Keras Tokenizer : 한국어 토큰 시퀀스를 정수 인덱스로 변환 (OOV 포함)
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(oov_token='<OOV>')      # 객체 생성(OOV 토큰 사용)
tokenizer.fit_on_texts(preprocessed_texts)    # 전처리된 토큰 리스트로 단어사전 학습 (word_index, index_word)
sequences = tokenizer.texts_to_sequences(preprocessed_texts)    # 문장(토큰 리스트)을 정수 인덱스 시퀀스로 변환
sequences

[[2, 4, 5], [6, 7, 8, 9, 3, 10, 11], [2, 12, 13, 14, 3, 15, 16, 17]]

In [ ]:
tokenizer.word_index  # 코퍼스에 등장한 단어(토큰) -> 인덱스 매핑 딕셔너리 (값이 작은 인덱스는 더 자주 등장한 단어)

{'<OOV>': 1,
 '오늘': 2,
 '하다': 3,
 '학원': 4,
 '간다': 5,
 '친구': 6,
 '맛있다': 7,
 '점심': 8,
 '식사': 9,
 '기분': 10,
 '좋다': 11,
 '어떻다': 12,
 '재미': 13,
 '수업': 14,
 '너무': 15,
 '기대': 16,
 '되다': 17}

In [ ]:
# Keras pad_sequences로 시퀀스 패딩 (길이 6)
from tensorflow.keras.preprocessing.sequence import pad_sequences

padded_seqs = pad_sequences(sequences, maxlen=6)  # 길이 6으로 통일 (기본값 padding = 'pre', truncating='pre')
padded_seqs

array([[ 0,  0,  0,  2,  4,  5],
       [ 7,  8,  9,  3, 10, 11],
       [13, 14,  3, 15, 16, 17]], dtype=int32)

In [16]:
padded_seqs.shape

(3, 6)

In [17]:
# 정수 시퀀스를 원-핫 벡터로 변환 (One-hot Encoding)
from tensorflow.keras.utils import to_categorical    # 정수 인덱스를 원-핫 벡터로 변환하는 함수

one_hot_encodded = to_categorical(padded_seqs)      # (문장, 길이) -> (문장, 길이, 클래스 수) 형태로 원-핫 인코딩
one_hot_encodded.shape

(3, 6, 18)

shape : (문장 개수, 패딩포함 길이, 최대인덱스 + 1)  
마지막 차원은 0(PAD)가 포함되어 +1이다.

In [18]:
# 원-핫 시퀀스 입력(길이 6, vocab 18)을 SimpleRNN 모델로 이진분류 구성
from tensorflow.keras import models, layers    # Keras 모델/레이어 구성 요소

input = layers.Input(shape=(6, 18))       # 입력 텐서 정의 : (timesteps=6, features=18) 원-핫 벡터 시퀀스
x = layers.SimpleRNN(8)(input)            # RNN 은닉유닛 8개
output = layers.Dense(1, activation='sigmoid')(x)  # 이진 분류용 출력: 확률(0~1) 1개로 변환

model = models.Model(inputs=input, outputs=output) # Functional API 로 입력 -> 출력 연결해 모델 생성
model.summary()                                    # 모델 레이어 구성/파라미터 수 요약 출력

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 6, 18)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 8)              │           216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 225 (900.00 B)

 Trainable params: 225 (900.00 B)

 Non-trainable params: 0 (0.00 B)

In [20]:
# 모델 컴파일 + 라벨 준비 + 원-핫 입력으로 학습 수행
import numpy as np

# 이진분류용 손실/옵티마이저/평가지표 학습 설정
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

labels = np.array([1, 0, 1])  # 각 샘플(문장)에 대한 정답 라벨

model.fit(one_hot_encodded, labels, epochs=3)

Epoch 1/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 687ms/step - accuracy: 0.3333 - loss: 0.8014
Epoch 2/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.6667 - loss: 0.7916
Epoch 3/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.6667 - loss: 0.7819


- **임베딩을 쓰기 전 단계 데모**로 가장 단순한 방법으로 학습까지 진행해봤다.
- 일반적으로 원- 핫 인코딩을
    - vocab이 커지면 원-핫 차원이 너무 커져서 **메모리/연산이 비효율적**
    - 단어 의미/유사도 정보를 못 담음
- 이러한 이유로 잘 사용하지 않기 때문에 정수 인덱스를 그대로 넣고 Embedding을 학습
    - 입력: `(batch, timesteps)` 정수 인덱스
    - `Embedding(vocab_size, embed_dim)` → `(batch, timesteps, embed_dim)`
    - 그 다음 RNN/Transformer 모델로 학습한다.

| 표현 방식                     | 입력 단위        | 벡터 형태(Shape)                               | 값이 의미하는 것                      | 순서(문맥) 보존    | 장점                                  | 단점                                      | 주 사용처                                     |
| ------------------------- | ------------ | ------------------------------------------ | ------------------------------ | ------------ | ----------------------------------- | --------------------------------------- | ----------------------------------------- |
| **원-핫(시퀀스)**              | 토큰 시퀀스(문장)   | `(seq_len, V)` 또는 배치 `(batch, seq_len, V)` | 각 토큰을 길이 `V` 벡터로 표시(해당 인덱스만 1) | ✅(시퀀스 구조 유지) | 구현/이해가 쉬움, RNN 입력으로 바로 가능           | `V`가 커지면 메모리/연산 폭발(매우 희소), 의미/유사도 정보 없음 | 교육용 데모, 아주 작은 vocab 실험                    |
| **BoW (CountVectorizer)** | 문서(문장/리뷰) 1개 | `(V)` 또는 `(batch, V)`                      | 단어 등장 **횟수**(count)            | ❌            | 빠르고 간단, 전통 ML에서 강력                  | 단어 순서/문맥 완전 손실, 고차원 희소                  | 스팸 분류/감성분석의 베이스라인, 빠른 EDA                 |
| **TF-IDF**                | 문서(문장/리뷰) 1개 | `(V)` 또는 `(batch, V)`                      | 단어 중요도 = 빈도(TF) × 희귀성(IDF)     | ❌            | BoW보다 “중요 단어”가 잘 드러남, 전통 ML 성능 좋음   | 문맥/순서 손실, 여전히 고차원 희소                    | 검색/유사도, 분류 베이스라인(Linear SVM, LR)          |
| **Embedding**             | 토큰 시퀀스(문장)   | `(seq_len, d)` 또는 `(batch, seq_len, d)`    | 토큰 ID → **d차원 실수 벡터**(학습/사전학습) | ✅            | 저차원 밀집(dense), 의미/유사도 학습 가능, 딥러닝 표준 | 학습 데이터/자원 필요, 해석이 상대적으로 어려움             | RNN/LSTM/GRU/Transformer 입력, 거의 모든 현대 NLP |